In [1]:
import cv2
import numpy as np
import os
from random import shuffle
from tqdm import tqdm
import tensorflow as tf
import matplotlib.pyplot as plt
%matplotlib inline

# !pip install q keras==2.3.1
from keras.models import Sequential
from keras.layers import  *
from keras.optimizers import *
from keras.models import load_model

Using TensorFlow backend.


In [2]:
classifier_directory = os.getcwd()
pathway_to_clips = classifier_directory + '/movie_clips'
pathway_to_frames = classifier_directory + '/frames'
pathway_to_skeletons = classifier_directory + '/skeletons'

parent_directory = classifier_directory[:-11]
webapp_directory = parent_directory + '/web_app'


train_data = classifier_directory + '/train'
test_data = classifier_directory + '/test'

def listdir_nohidden(directory):
    filelist = os.listdir(directory)
    return [x for x in filelist
            if not (x.startswith('.'))]

def move_label(img): 
    label = img.split('.')[0]
    if label == 'turn':
        ohl = np.array([1,0,0])
    elif label == 'cuddle':
        ohl = np.array([0,1,0])
    elif label == 'shadow':
        ohl = np.array([0,0,1])
    return ohl

def train_data_with_label():
    train_images = []
    for ii in tqdm(listdir_nohidden(train_data)):
        path = os.path.join(train_data, ii)
        img = 1-np.absolute(np.genfromtxt(path, delimiter=',')) #cv2.imread(path)
        train_images.append([np.array(img), move_label(ii)])
    shuffle(train_images)
    return train_images

def test_data_with_label():
    test_images = []
    for ii in tqdm(listdir_nohidden(test_data)):
        path = os.path.join(test_data, ii)
        img = 1-np.absolute(np.genfromtxt(path, delimiter=',')) 
        test_images.append([np.array(img), move_label(ii)])
    shuffle(test_images)
    return test_images
                         
                         
                         
training_images = train_data_with_label()
testing_images = test_data_with_label()


tr_img_data = np.array([ii[0] for ii in training_images]).reshape(-1,36,60,1)
tr_lbl_data = np.array([ii[1] for ii in training_images])

tst_img_data = np.array([ii[0] for ii in testing_images]).reshape(-1,36,60,1)
tst_lbl_data = np.array([ii[1] for ii in testing_images])
                         
       

100%|██████████| 58/58 [00:00<00:00, 326.75it/s]


In [3]:

# model = Sequential()
# model.add(Conv2D(filters=60, kernel_size=[36,4],strides=[36,2],padding='same', activation='relu'))
# model.add(Activation('relu'))
# model.add(MaxPooling2D(pool_size=(36, 2), padding='same'))

# model.add(Conv2D(filters=60, kernel_size=[18,4],strides=[18,2],padding='same', activation='relu'))
# model.add(Activation('relu'))
# model.add(MaxPooling2D(pool_size=(18, 2), padding='same'))

# model.add(Conv2D(filters=60, kernel_size=[9,4],strides=[9,2],padding='same', activation='relu'))
# model.add(Activation('relu'))
# model.add(MaxPooling2D(pool_size=(9, 2), padding='same'))

# model.add(Flatten())
# model.add(Dense(36))
# model.add(Activation('relu'))
# model.add(Dropout(0.5))
# model.add(Dense(9))
# model.add(Activation('softmax'))

model = Sequential()

model.add(InputLayer(input_shape = [36,60,1]))

model.add(Conv2D(filters=60, kernel_size=[36,4],strides=[36,2],padding='same', activation='softmax'))
model.add(MaxPool2D(pool_size=4, padding='same'))

model.add(Conv2D(filters=60, kernel_size=[18,4],strides=[18,2],padding='same', activation='softmax'))
model.add(MaxPool2D(pool_size=4, padding='same'))

model.add(Conv2D(filters=60, kernel_size=[9,4],strides=[9,2],padding='same', activation='softmax'))
model.add(MaxPool2D(pool_size=4, padding='same'))

model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(512,activation='softmax'))
model.add(Dropout(rate=0.5))
model.add(Dense(3,activation='softmax'))
optimizer = Adam(lr=1e-3)

model.compile(optimizer=optimizer, loss='categorical_crossentropy',metrics=['accuracy'])
model.fit(x=tr_img_data,y=tr_lbl_data,epochs=50,batch_size=100)
# model.summary()


Epoch 1/50
146/146 [==============================] - 0s 2ms/step - loss: 1.0971 - accuracy: 0.5548
Epoch 2/50
146/146 [==============================] - 0s 209us/step - loss: 1.0960 - accuracy: 0.5822
Epoch 3/50
146/146 [==============================] - 0s 214us/step - loss: 1.0940 - accuracy: 0.5890
Epoch 4/50
146/146 [==============================] - 0s 216us/step - loss: 1.0920 - accuracy: 0.5890
Epoch 5/50
146/146 [==============================] - 0s 210us/step - loss: 1.0902 - accuracy: 0.5890
Epoch 6/50
146/146 [==============================] - 0s 207us/step - loss: 1.0883 - accuracy: 0.5890
Epoch 7/50
146/146 [==============================] - 0s 210us/step - loss: 1.0860 - accuracy: 0.5890
Epoch 8/50
146/146 [==============================] - 0s 210us/step - loss: 1.0842 - accuracy: 0.5890
Epoch 9/50
146/146 [==============================] - 0s 208us/step - loss: 1.0821 - accuracy: 0.5890
Epoch 10/50
146/146 [==============================] - 0s 211us/step - loss: 1.0801 

In [4]:
scores = model.evaluate(tst_img_data, tst_lbl_data, verbose=0)
print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

accuracy: 51.72%


In [5]:
# model.save("model_.h5")

# Save the weights

model.save_weights(webapp_directory + '/model_weights.h5')

# # Save the model architecture
# with open('model_architecture.json', 'w') as f:
#     f.write(model.to_json())